# ChatGPT 3.5

# Can ChatGPT detect smishing?
In this experiment, we evaluate the capability of ChatGPT 3.5 in detecting smishing messages.
Using the test dataset (refer to 'smishing_data.ipynb'), we prompt ChatGPT to assess whether each message is a smish or ham.

In [1]:
from openai import OpenAI
import json
import pickle
import time
client = OpenAI()

### ChatGPT's understanding of the terms "smish" and "ham".

First, we check if ChatGPT understands the term "smish message".

In [2]:
smish = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a cybersecurity researcher."},
    {"role": "user", "content": "Provide a brief definition of a 'smish' message in the context of SMS security."}
  ]
)

In [3]:
smish.choices[0].message.content

"A 'smish' message, also known as SMS phishing, is a type of cyber attack where attackers send deceptive text messages to trick individuals into revealing sensitive information or performing harmful actions. These messages often appear as legitimate notifications from banks, service providers, or organizations, aiming to manipulate users into clicking on malicious links, downloading malware, or sharing personal information such as passwords or financial data. Smishing attacks exploit the trust and familiarity associated with text messages to deceive and compromise victims."

Similarily, we check if ChatGPT understands the term "ham message".

In [4]:
ham = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a cybersecurity researcher."},
    {"role": "user", "content": "Provide a brief definition of a 'ham' message in the context of SMS security."}
  ]
)

In [5]:
ham.choices[0].message.content

'In the context of SMS security, a \'ham\' message refers to a legitimate, non-malicious message that is incorrectly identified as a spam or phishing message by a spam or fraud detection system. The term \'ham\' is derived from the acronym HAM, which stands for "Harmless Advertising Message." Ham messages are typically benign messages such as promotional offers, notifications, or personal communication that are mistakenly flagged as spam. These false positives can occur due to various factors, including the use of certain keywords, sender reputation, or other characteristics that spam filters use to determine the legitimacy of a message.'

### ChatGPT-based smishing detection.

In this experiment, we prompt ChatGPT: "Do you think it is a ham or a smish message?".

This essential question is followed by some sentences that coerce ChatGPT to output only a single word "ham" or "smish":

"Do you think it is a ham or smish message? Your output should be a single word 'smish' or 'ham'. Do not write a sentence. Output is case-sensitive."

We need it for automation purposes.

In [6]:
# test data
with open("./data/test_data.pkl", "rb") as input_file:
    test_data = pickle.load(input_file)

In [7]:
X_test = test_data["X_test"]
y_test = test_data["y_test"]

total_hams_count = 0
total_smishes_count = 0

for label in y_test:
    if label == "ham":
        total_hams_count += 1
    if label == "smish":
        total_smishes_count += 1

In [8]:
print("There is {} hams and {} smishes in the test dataset.".format(total_hams_count, total_smishes_count))

There is 954 hams and 161 smishes in the test dataset.


In [9]:
false_hams_indicies = []
false_smishes_indicies = []
false_hams_count = 0
false_smishes_count = 0
true_hams_count = 0
true_smishes_count = 0
errors_count = 0
errors_indicies = []
errors = []


for i in range(len(X_test)):
    prompt = """Do you think it is a ham or smish message?
    Your output should be a single word 'smish' or 'ham'.
    Do not write a sentence.
    Output is case-sensitive.
    
    {}
    """.format(X_test[i])

    completion = client.chat.completions.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", 
         "content": prompt
        }
      ]
    )

    answer = completion.choices[0].message.content

    if answer not in ["ham", "smish"]:
        errors_count += 1
        errors_indicies.append(i)
        errors.append(answer)
        continue
    elif answer == "ham" and y_test[i] == "ham": # correctly recognized as a ham
        true_hams_count += 1
    elif answer == "smish" and y_test[i] == "smish": # correctly recognized as a smish
        true_smishes_count += 1
    elif answer == "ham" and y_test[i] == "smish": # wrongly recognized as a ham
        false_hams_indicies.append(i)
        false_hams_count += 1
    elif answer == "smish" and y_test[i] == "ham": # wrongly recognized as a smish
        false_smishes_indicies.append(i)
        false_smishes_count += 1

    time.sleep(0.5) # because of API limits (500 requests per minute and 60 000 tokens per minute)
        
# errors warning   
if errors_count != 0:
    if errors_count == 1:
        print("WARNING: {} error".format(errors_count))
    else:
        print("WARNING: {} errors".format(errors_count))

# save results for further analysis
results = {"FN" : false_hams_count, "FP" : false_smishes_count, 
           "TN" : true_hams_count, "TP" : true_smishes_count,
           "FN_indicies" : false_hams_indicies, "FP_indicies" : false_smishes_indicies,
            "errors_count" : errors_count, "errors" : errors, "errors_indicies" : errors_indicies}

with open("./results/results_chatGPT_3_5.pkl", 'wb') as handle:
    pickle.dump(results, handle, protocol=pickle.HIGHEST_PROTOCOL)

### The performance of ChatGPT in smishing detection.

In [10]:
FN = false_hams_count     #FN - messages wrongly recognized as not smishes (hams)
FP = false_smishes_count  #FP - messages wrongly recognized as smishes
TN = true_hams_count      #TN - messages correctly recognized as not smishes (hams)
TP = true_smishes_count   #TP - messages correctly recognized as smishes
TOTAL = FN + FP + TN + TP

In [11]:
print("Messages wrongly recognized as hams: {0:.2f}%".format(FN / TOTAL * 100))
print("Messages wrongly recognized as smishes: {0:.2f}%".format(FP / TOTAL * 100))
print("Messages correctly recognized as hams: {0:.2f}%".format(TN / TOTAL * 100))
print("Messages correctly recognized as smishes: {0:.2f}%".format(TP / TOTAL * 100))

Messages wrongly recognized as hams: 3.96%
Messages wrongly recognized as smishes: 27.27%
Messages correctly recognized as hams: 58.51%
Messages correctly recognized as smishes: 10.26%


#### Accuracy

In [12]:
accuracy = (TP + TN) / TOTAL
print("{0:.2f}%".format(accuracy * 100))

68.77%


#### Recall

In [13]:
recall = TP / (TP + FN)
print("{0:.2f}%".format(recall * 100))

72.15%


#### Precision

In [14]:
precision = TP / (TP + FP)
print("{0:.2f}%".format(precision * 100))

27.34%


#### F1 score

In [15]:
F1_score = TP / (TP + (FP + FN) / 2)
print("{0:.2f}%".format(F1_score * 100))

39.65%
